In [1]:
import joblib

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
import re

def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet

In [2]:
df = pd.read_csv("../data/hate_classes.tsv", sep='\t')
# process text for embeddings
text = df.Text.apply(basic_tokenize)

tokenizer = Tokenizer(num_words=20000, split=' ', oov_token='<unw>', filters='')
tokenizer.fit_on_texts(text.values)
X = tokenizer.texts_to_sequences(text.values)
X = pad_sequences(X, maxlen=200)

voc = tokenizer.word_index
print('Found %s unique tokens.' % len(voc))

embeddings_index = {}
f = open(os.path.join('./', 'glove.twitter.27B.200d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))


/home/guoanhu/.pyenv/versions/3.6.14/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Found 20965 unique tokens.
Found 1193515 word vectors.


In [3]:
max_features = 20000
num_words = min(max_features, len(voc))
print(num_words)

embedding_dim = 200

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in voc.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

20000


In [4]:
Y = []
for row in df.itertuples():
    Y.append((row.RAE,row.NAT,row.SXO,row.GEN,row.REL,row.IDL,row.POL,row.MPH))
Y = np.array(Y)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, Embedding, LSTM
from tensorflow.keras.optimizers import RMSprop
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=200, weights=[embedding_matrix], input_length=200, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True , recurrent_dropout = 0.25 , dropout = 0.25))
model.add(LSTM(units=64 , recurrent_dropout = 0.1 , dropout = 0.1))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(8, activation='sigmoid'))
model.compile(optimizer=RMSprop(lr = 0.001), loss='categorical_crossentropy', metrics=['binary_accuracy'])
model.summary()


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 200)          4000000   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 128)          168448    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
Total params: 4,220,200
Trainable params: 220,200
Non-t

In [6]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import RMSprop
model = Sequential()
model.add(Embedding(max_features, output_dim=200, weights=[embedding_matrix], input_length=200, trainable=False))
model.add(Conv1D(filters=64, kernel_size=8, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(40, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
# compile network
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = 0.001), metrics=['binary_accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 200)          4000000   
_________________________________________________________________
conv1d (Conv1D)              (None, 193, 64)           102464    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 96, 64)            0         
_________________________________________________________________
flatten (Flatten)            (None, 6144)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 40)                245800    
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 328       
Total params: 4,348,592
Trainable params: 348,592
Non-trainable params: 4,000,000
______________________________________

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.1)

batch_size = 64  # mini-batch with 256 examples
epochs = 30
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 4371 samples, validate on 486 samples
Epoch 1/30
4371/4371 [==============================] - 4s 807us/sample - loss: 2.2835 - binary_accuracy: 0.5147 - val_loss: 2.2571 - val_binary_accuracy: 0.4954
Epoch 2/30
4371/4371 [==============================] - 0s 93us/sample - loss: 2.1161 - binary_accuracy: 0.6740 - val_loss: 2.2126 - val_binary_accuracy: 0.7623
Epoch 3/30
4371/4371 [==============================] - 0s 88us/sample - loss: 1.8120 - binary_accuracy: 0.8433 - val_loss: 2.3265 - val_binary_accuracy: 0.8413
Epoch 4/30
4371/4371 [==============================] - 0s 84us/sample - loss: 1.4678 - binary_accuracy: 0.8639 - val_loss: 2.7073 - val_binary_accuracy: 0.8467
Epoch 5/30
4371/4371 [==============================] - 0s 85us/sample - loss: 1.2074 - binary_accuracy: 0.8613 - val_loss: 2.6245 - val_binary_accuracy: 0.8467
Epoch 6/30
4371/4371 [==============================]

In [8]:
x_test[1].shape

(200,)

In [25]:
a = model.predict(x_test[1].reshape(1,200))
a = np.round(a)
print(df.loc[1,:].Text)
print(a)
print(y_test[1])
# (row.RAE,row.NAT,row.SXO,row.GEN,row.REL,row.IDL,row.POL,row.MPH)
model.save('../models/hate_classification.h5')

“These [companies] are run by sociopaths,” he said. “These people are complete narcissists. These people ought to be controlled, they ought to be regulated.” At one point during the phone call, Bannon said, “These people are evil.' https://www.vanityfair.com/news/2018/08/steve-bann...
[[1. 0. 0. 0. 0. 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 1. 0.]
